In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Parameters

In [3]:
# Fixed parameters

DEVICE = 'ibm_torino'
_RESETS = False

backend = provider.get_backend(DEVICE)

In [4]:
# Variable parameters
import numpy as np

logicals = ['0', '1']
x_basis = [True, False]

rounds = [10, 20, 30, 40, 50, 75, 100]

print("rounds:", rounds)

rounds: [10, 20, 30, 40, 50, 75, 100]


# Find the best layout

In [15]:
from soft_info import BackendEvaluator

evaluator = BackendEvaluator(backend)
longest_path, _, _, path_info = evaluator.find_longest_good_RepCode_string(longest_length=111)

Trying RepCode string of length 111 => distance 56...


In [8]:
path_info

{'mean_gate_error': 0.00636934970598819,
 'min_gate_error': 0.0016040680953005315,
 'max_gate_error': 0.03133666302510113,
 'mean_readout_error': 0.02837927927927928,
 'min_readout_error': 0.006199999999999983,
 'max_readout_error': 0.1763,
 'mean_ancilla_error': 0.035709090909090906,
 'min_ancilla_error': 0.00869999999999993,
 'max_ancilla_error': 0.10729999999999995}

In [9]:
link_qubits = longest_path[1::2]
code_qubits = longest_path[0::2]
layout = link_qubits + code_qubits
distance = len(code_qubits)
print("longest path:", longest_path)
print("layout:", layout)
print("distance:", distance)

longest path: [13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0, 15, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 37, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 53, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 75, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 91, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 113, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 129]
layout: [12, 10, 8, 6, 4, 2, 0, 19, 21, 23, 25, 27, 29, 31, 33, 52, 50, 48, 46, 44, 42, 40, 38, 57, 59, 61, 63, 65, 67, 69, 71, 90, 88, 86, 84, 82, 80, 78, 76, 95, 97, 99, 101, 103, 105, 107, 109, 128, 126, 124, 122, 120, 118, 116, 114, 13, 11, 9, 7, 5, 3, 1, 15, 20, 22, 24, 26, 28, 30, 32, 37, 51, 49, 47, 45, 43, 41, 39, 53, 58, 60, 62, 64, 66, 68, 70, 75, 89, 87, 85, 83, 81, 79, 77, 91, 96, 98, 100, 102, 104, 106, 108, 113, 127, 125, 123, 121, 119, 117, 115, 129]
distance: 56


# No rounds loop

In [10]:
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import run_IQ_calibration
from qiskit import transpile
from Scratch import metadata_helper
from tqdm import tqdm


############################

# Round index
round_index = 0

# Z = 0
xbasis = False
logical = '0'

############################

T = rounds[round_index]
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)


12:50:17 Warning: updating metadata. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/libs/qiskit-saver-provider/result_saver/provider/saver_provider.py, LINE: 46


100%|██████████| 20/20 [00:49<00:00,  2.48s/it]


In [11]:
############################

# Z = 1
xbasis = False
logical = '1'

############################

T = rounds[round_index]
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

100%|██████████| 20/20 [00:50<00:00,  2.51s/it]


In [12]:
############################

# X = 0
xbasis = True
logical = '0'

############################

T = rounds[round_index]
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

100%|██████████| 20/20 [00:49<00:00,  2.46s/it]


In [13]:
############################

# X = 1
xbasis = True
logical = '1'

############################

T = rounds[round_index]
REPETITIONS = 20

code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
qc = code.circuit[logical]

# Add repdelay at the end of the circuit 
range_rd = backend.configuration().rep_delay_range
qc.barrier()
qc.delay(unit='s', duration=4*range_rd[1])
qc.barrier()

# Transpile and run
transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 
n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                            distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                            path_info=path_info)

# Run calibration
calib_shots = int(4e6/backend.configuration().n_qubits)
run_IQ_calibration(backend, shots=calib_shots)


for _ in tqdm(range(REPETITIONS)):
        backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
                job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])
        
# Run calibration
run_IQ_calibration(backend, shots=calib_shots)

100%|██████████| 20/20 [01:40<00:00,  5.01s/it]


# Round loop!

In [30]:

# Z = 0
xbasis = False
logical = '0'

# Run calibration

for T in rounds:
    code = RepetitionCodeCircuit(distance, T, xbasis=xbasis)
    qc = code.circuit[logical]
    transpiled_qc = transpile(qc, backend, initial_layout=layout, optimization_level=1) 

    n_shots = int(4e6/((distance-1)*T+2*distance-1)) # Exact shot formula to have exactly 4M shots per job
    metadata = metadata_helper(descr='subset RepCodes', code="RepetitionCodeCircuit", 
                               distance=distance, rounds=T, logical=logical, xbasis=xbasis,
                               path_info=path_info)

    # backend.run(metadata, transpiled_qc, shots=n_shots, meas_level=1, meas_return='single', 
    #             job_tags=[f'Subset {distance}, {T} rounds, {logical} log, xbasis={xbasis}'])

# Run calibration
    

{'descr': 'subset RepCodes', 'code': 'RepetitionCodeCircuit', 'distance': 52, 'rounds': 10, 'logical': '0', 'xbasis': False, 'path_info': {'mean_gate_error': 0.007963147793478936, 'min_gate_error': 0.0034766228550469347, 'max_gate_error': 0.0232547718972346, 'mean_readout_error': 0.022770873786407766, 'min_readout_error': 0.0019000000000000128, 'max_readout_error': 0.3568, 'mean_ancilla_error': 0.02186078431372549, 'min_ancilla_error': 0.0019000000000000128, 'max_ancilla_error': 0.24750000000000005}}
{'descr': 'subset RepCodes', 'code': 'RepetitionCodeCircuit', 'distance': 52, 'rounds': 20, 'logical': '0', 'xbasis': False, 'path_info': {'mean_gate_error': 0.007963147793478936, 'min_gate_error': 0.0034766228550469347, 'max_gate_error': 0.0232547718972346, 'mean_readout_error': 0.022770873786407766, 'min_readout_error': 0.0019000000000000128, 'max_readout_error': 0.3568, 'mean_ancilla_error': 0.02186078431372549, 'min_ancilla_error': 0.0019000000000000128, 'max_ancilla_error': 0.24750000